<a href="https://colab.research.google.com/github/nezihaksu/Airbnb_Istanbul/blob/main/airbnb_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from sklearn.impute import SimpleImputer
import re

#Explatory Data Analysis

In [24]:
DF = r'/content/drive/MyDrive/listings.csv'
FILE_TYPE = "csv"
IMPUTE = True
ALLOWED_NAN_PERCENTAGE = 10
DROP_KEYWORDS = ["code","zipcode","link","url","id","name","thumbnail","picture","pic","description","note"]
NONE_VALUES = [np.nan,None,"none","null","nan",""," ",0]


class Explore():
  """Explore the dataset."""
  def __init__(self,df,file_type:str):
    if file_type == "xlsx" or  file_type == "xls":
      self.df = pd.read_excel(df,engine="python")
    self.df = pd.read_csv(df,engine="python")
    self.file_type = file_type

  def __call__(self):
    return self.df

  def intro(self):
    return "===INFO===",self.df.info(),"===DESCRIPTION===",self.df.describe(),"===DTYPES==",self.df.dtypes
  
  def unique_values(self):
    #Unique values that are in features.
    for column in self.df.columns:
      print(column.upper()+ " UNIQUE VALUES")
      print(str(df[column].unique())+"\n")

  def missing_values(self):
	  missing_percentage = self.df.isnull().sum()*100/len(self.df)
	  plt.figure(figsize=(5, 15))
	  missing_percentage.plot(kind='barh')
	  plt.xticks(rotation=90, fontsize=10)
	  plt.yticks(fontsize=5)
	  plt.xlabel("Missing Percentage", fontsize=14)
	  plt.show()
   
  #Plotting histograms of the numerical features to see the distribution of each of them.
  def dtype_histogram(self,data_type:str):
    numerical_features = self.df.dtypes[self.df.dtypes == data_type].index.to_list()
    self.df[numerical_features].hist(bins = 50,figsize = (20,15))
    plt.show()

In [25]:
explore = Explore(df=DF,file_type=FILE_TYPE)

#Cleaning Dataset

In [27]:
class Cleaner():
  """Clean the dataset."""
  def __init__(self,df,file_type:str):
    if file_type == "xlsx" or  file_type == "xls":
      self.df = pd.read_excel(df,engine="python")
    self.df = pd.read_csv(df,engine="python")
    self.file_type = file_type

  def _drop_type_column(self,pattern:str,inplace:bool):
    for column in self.df.columns:
      if any(self.df[column].astype(str).str.contains(pattern,regex=True)):
        self.df.drop(column,axis=1,inplace=inplace)
    return self.df


  #Expanding one column dataframe into multiple columns according to split character.
  def split_column_into_df(self,column_index:int,split_char:str):
    if len(df.columns) == 1:
      quotes_strip = list(self.df.columns)[0].replace(strip_char,'')
      columns_split = quotes_strip.split(split_char)
      self.df = self.df[self.df.iloc[:,0].name].str.split(pat = split_char,expand = True)
      self.df.columns =  columns_split
      self.df.replace(split_char,'',regex = True,inplace = True)
    print("This method is only for explanding single column dataframes!")
    return self.df

  def drop_missing_columns(self,percentage):
    missing_percentage = self.df.isnull().sum()*100/len(self.df)
    features_left = missing_percentage[missing_percentage < percentage].index
    return self.df[features_left]

  #Drop columns by their names.
  def drop_column_contains(self,keywords:list):
    for keyword in keywords:
      keyword_pattern = re.compile(keyword)
      for column in self.df.columns:
        if keyword_pattern.search(column):
          self.df.drop(column,axis=1,inplace=True)
    return self.df

  def drop_text_columns(self,inplace):
    sentence_pattern = r'[A-z][A-z]+?\W'
    link_pattern = r'[A-z][A-z]+?://'
    text_pattern = r'|'.join((sentence_pattern,link_pattern))  
    return self._drop_type_column(text_pattern,inplace)
  
  
  def drop_date_columns(self,inplace:bool):
    date_pattern_dash = r"([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))"
    date_pattern_dot = r"([12]\d{3}.(0[1-9]|1[0-2]).(0[1-9]|[12]\d|3[01]))"
    date_pattern_slash = r"([12]\d{3}/(0[1-9]|1[0-2])/(0[1-9]|[12]\d|3[01]))"
    date_pattern_dash_text =  r"([12]\d{3}-([A-z]+)-(0[1-9]|[12]\d|3[01]))"
    date_pattern_dot_text = r"([12]\d{3}.([A-z]+).(0[1-9]|[12]\d|3[01]))"
    date_pattern_slash_text = r"([12]\d{3}/([A-z]+)/(0[1-9]|[12]\d|3[01]))"
    date_pattern = r'|'.join((date_pattern_dash,
                              date_pattern_dot,
                              date_pattern_slash,
                              date_pattern_dash_text,
                              date_pattern_dot_text,
                              date_pattern_slash_text))
    return self._drop_type_column(date_pattern,inplace)
  
  def impute_nan_values(self,strategy="most_frequent"):
    simple_imputer = SimpleImputer(strategy=strategy)
    for column in df.columns:
      if pd.DataFrame.any(df[column].isnull()):
        df[column] = simple_imputer.fit_transform(df[column])
       


In [28]:
cleaner = Cleaner(DF,FILE_TYPE)

In [30]:
cleaner.drop_column_contains(DROP_KEYWORDS)

,experiences_offered,transit,access,interaction,house_rules,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,market,smart_location,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,none,NaN,NaN,NaN,NaN,"Istanbul, Istanbul, Turkey",NaN,NaN,NaN,f,Üsküdar,1.0,1.0,"['email', 'phone', 'facebook', 'reviews']","Istanbul Province, Istanbul, Turkey",Üsküdar,Uskudar,NaN,Istanbul Province,Istanbul,Istanbul,"Istanbul Province, Turkey",Turkey,41.05650,29.05367,f,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Smoking ...",700.0,$624.00,NaN,NaN,NaN,NaN,2,$154.00,1,730,1,1,730,730,1.0,730.0,24 months ago,t,30,60,90,365,1,0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,flexible,f,1,1,0,0,0.01
1,none,"The city center, Taksim is about 6-7 kilometer...",Our dear guests may enjoy the oven and the was...,"Depending on our time available, we may even t...",- Windows facing the street should not be left...,"Ankara, Ankara, Turkey",NaN,NaN,NaN,f,Beşiktaş,2.0,2.0,"['email', 'phone', 'facebook', 'reviews', 'wor...","Istanbul, Turkey",Beşiktaş,Besiktas,NaN,Istanbul,NaN,Istanbul,"Istanbul, Turkey",Turkey,41.06984,29.04545,t,Apartment,Entire home/apt,3,1.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",""Air conditioning"",Kitchen,""Fre...",NaN,$113.00,"$1,348.00","$4,615.00",$588.00,NaN,4,$208.00,30,210,30,30,210,210,30.0,210.0,3 months ago,t,0,0,0,117,41,5,90.0,9.0,9.0,10.0,10.0,10.0,9.0,f,NaN,f,f,moderate,f,2,1,1,0,0.37
2,none,"Otobus , Vapur , Metro , taksi",NaN,Yes I can arrange it,NaN,"Istanbul, Istanbul, Turkey",NaN,NaN,NaN,f,Beşiktaş,1.0,1.0,"['email', 'phone', 'offline_government_id', 's...","Beşiktaş/ bebek, İstanbul, Turkey",Beşiktaş,Besiktas,NaN,Beşiktaş/ bebek,İstanbul,Istanbul,"Beşiktaş/ bebek, Turkey",Turkey,41.07731,29.03891,t,Serviced apartment,Entire home/apt,6,1.0,2.0,5.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",969.0,$232.00,"$2,980.00","$9,934.00","$19,869.00",$0.00,1,$0.00,21,1125,21,21,1125,1125,21.0,1125.0,2 weeks ago,t,0,0,0,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,t,f,strict_14_with_grace_period,f,1,1,0,0,NaN
3,none,All is within close walking distance. Also sit...,NaN,I will be away from Istanbul during your stay ...,This is the place I am living. Not like the pl...,"Istanbul, Istanbul, Turkey",NaN,NaN,NaN,f,Cihangir,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...","Istanbul Province, Turkey",Cihangir,Beyoglu,NaN,Istanbul Province,NaN,Istanbul,"Istanbul Province, Turkey",Turkey,41.03220,28.98216,f,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",NaN,$267.00,"$1,523.00","$5,431.00",$662.00,$265.00,2,$99.00,5,90,5,5,90,90,5.0,90.0,3 months ago,t,0,8,8,277,2,1,100.0,10.0,9.0,10.0,10.0,10.0,10.0,f,NaN,f,f,moderate,f,1,1,0,0,0.04
4,none,NaN,NaN,NaN,NaN,"Istanbul, Turkey",within an hou

In [31]:
cleaner.drop_text_columns(inplace=True)

,experiences_offered,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,none,NaN,NaN,f,1.0,1.0,Üsküdar,Uskudar,NaN,41.05650,29.05367,f,2,1.0,0.0,1.0,700.0,$624.00,NaN,NaN,NaN,NaN,2,$154.00,1,730,1,1,730,730,1.0,730.0,t,30,60,90,365,1,0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,flexible,f,1,1,0,0,0.01
1,none,NaN,NaN,f,2.0,2.0,Beşiktaş,Besiktas,NaN,41.06984,29.04545,t,3,1.0,2.0,2.0,NaN,$113.00,"$1,348.00","$4,615.00",$588.00,NaN,4,$208.00,30,210,30,30,210,210,30.0,210.0,t,0,0,0,117,41,5,90.0,9.0,9.0,10.0,10.0,10.0,9.0,f,NaN,f,f,moderate,f,2,1,1,0,0.37
2,none,NaN,NaN,f,1.0,1.0,Beşiktaş,Besiktas,NaN,41.07731,29.03891,t,6,1.0,2.0,5.0,969.0,$232.00,"$2,980.00","$9,934.00","$19,869.00",$0.00,1,$0.00,21,1125,21,21,1125,1125,21.0,1125.0,t,0,0,0,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,t,f,strict_14_with_grace_period,f,1,1,0,0,NaN
3,none,NaN,NaN,f,1.0,1.0,Cihangir,Beyoglu,NaN,41.03220,28.98216,f,2,1.0,1.0,1.0,NaN,$267.00,"$1,523.00","$5,431.00",$662.00,$265.00,2,$99.00,5,90,5,5,90,90,5.0,90.0,t,0,8,8,277,2,1,100.0,10.0,9.0,10.0,10.0,10.0,10.0,f,NaN,f,f,moderate,f,1,1,0,0,0.04
4,none,100%,NaN,f,19.0,19.0,Şişli,Sisli,NaN,41.04471,28.98567,t,5,1.0,1.0,3.0,753.0,$654.00,NaN,NaN,NaN,$66.00,2,$66.00,3,360,3,3,360,360,3.0,360.0,t,12,34,45,302,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,t,f,moderate,f,19,19,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,none,NaN,NaN,f,1.0,1.0,NaN,Bahcelievler,NaN,40.99535,28.84067,t,4,NaN,1.0,2.0,NaN,$107.00,NaN,NaN,$0.00,$30.00,1,$0.00,3,1125,3,3,1125,1125,3.0,1125.0,t,29,59,89,269,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,t,f,flexible,f,1,0,1,0,NaN
17324,none,100%,NaN,f,1.0,1.0,Şişli,Sisli,NaN,41.05814,28.99161,f,3,1.0,2.0,2.0,NaN,$303.00,NaN,NaN,NaN,NaN,1,$0.00,3,15,3,3,15,15,3.0,15.0,t,23,29,41,131,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,flexible,f,1,1,0,0,NaN
17325,none,NaN,NaN,f,1.0,1.0,NaN,Maltepe,NaN,40.95070,29.12381,f,1,1.0,1.0,1.0,NaN,$148.00,NaN,NaN,NaN,NaN,1,$0.00,1,5,1,1,5,5,1.0,5.0,t,0,7,33,33,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,flexible,f,1,0,1,0,NaN
17326,none,NaN,NaN,f,1.0,1.0,Beşiktaş,Besiktas,NaN,41.07760,29.02340,t,2,1.0,1.0,1.0,NaN,$148.00,NaN,NaN,NaN,NaN,1,$0.00,1,7,1,1,7,7,1.0,7.0,t,29,49,79,115,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,moderate,f,1,0,1,0,NaN


In [32]:
cleaner.drop_date_columns(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,experiences_offered,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,none,NaN,NaN,f,1.0,1.0,Üsküdar,Uskudar,NaN,41.05650,29.05367,f,2,1.0,0.0,1.0,700.0,$624.00,NaN,NaN,NaN,NaN,2,$154.00,1,730,1,1,730,730,1.0,730.0,t,30,60,90,365,1,0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,flexible,f,1,1,0,0,0.01
1,none,NaN,NaN,f,2.0,2.0,Beşiktaş,Besiktas,NaN,41.06984,29.04545,t,3,1.0,2.0,2.0,NaN,$113.00,"$1,348.00","$4,615.00",$588.00,NaN,4,$208.00,30,210,30,30,210,210,30.0,210.0,t,0,0,0,117,41,5,90.0,9.0,9.0,10.0,10.0,10.0,9.0,f,NaN,f,f,moderate,f,2,1,1,0,0.37
2,none,NaN,NaN,f,1.0,1.0,Beşiktaş,Besiktas,NaN,41.07731,29.03891,t,6,1.0,2.0,5.0,969.0,$232.00,"$2,980.00","$9,934.00","$19,869.00",$0.00,1,$0.00,21,1125,21,21,1125,1125,21.0,1125.0,t,0,0,0,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,t,f,strict_14_with_grace_period,f,1,1,0,0,NaN
3,none,NaN,NaN,f,1.0,1.0,Cihangir,Beyoglu,NaN,41.03220,28.98216,f,2,1.0,1.0,1.0,NaN,$267.00,"$1,523.00","$5,431.00",$662.00,$265.00,2,$99.00,5,90,5,5,90,90,5.0,90.0,t,0,8,8,277,2,1,100.0,10.0,9.0,10.0,10.0,10.0,10.0,f,NaN,f,f,moderate,f,1,1,0,0,0.04
4,none,100%,NaN,f,19.0,19.0,Şişli,Sisli,NaN,41.04471,28.98567,t,5,1.0,1.0,3.0,753.0,$654.00,NaN,NaN,NaN,$66.00,2,$66.00,3,360,3,3,360,360,3.0,360.0,t,12,34,45,302,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,t,f,moderate,f,19,19,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,none,NaN,NaN,f,1.0,1.0,NaN,Bahcelievler,NaN,40.99535,28.84067,t,4,NaN,1.0,2.0,NaN,$107.00,NaN,NaN,$0.00,$30.00,1,$0.00,3,1125,3,3,1125,1125,3.0,1125.0,t,29,59,89,269,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,t,f,flexible,f,1,0,1,0,NaN
17324,none,100%,NaN,f,1.0,1.0,Şişli,Sisli,NaN,41.05814,28.99161,f,3,1.0,2.0,2.0,NaN,$303.00,NaN,NaN,NaN,NaN,1,$0.00,3,15,3,3,15,15,3.0,15.0,t,23,29,41,131,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,flexible,f,1,1,0,0,NaN
17325,none,NaN,NaN,f,1.0,1.0,NaN,Maltepe,NaN,40.95070,29.12381,f,1,1.0,1.0,1.0,NaN,$148.00,NaN,NaN,NaN,NaN,1,$0.00,1,5,1,1,5,5,1.0,5.0,t,0,7,33,33,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,flexible,f,1,0,1,0,NaN
17326,none,NaN,NaN,f,1.0,1.0,Beşiktaş,Besiktas,NaN,41.07760,29.02340,t,2,1.0,1.0,1.0,NaN,$148.00,NaN,NaN,NaN,NaN,1,$0.00,1,7,1,1,7,7,1.0,7.0,t,29,49,79,115,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,f,f,moderate,f,1,0,1,0,NaN


In [33]:
cleaner.drop_missing_columns(percentage=10)

,experiences_offered,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,price,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,none,f,1.0,1.0,Uskudar,41.05650,29.05367,f,2,1.0,0.0,1.0,$624.00,2,$154.00,1,730,1,1,730,730,1.0,730.0,t,30,60,90,365,1,0,f,f,f,flexible,f,1,1,0,0
1,none,f,2.0,2.0,Besiktas,41.06984,29.04545,t,3,1.0,2.0,2.0,$113.00,4,$208.00,30,210,30,30,210,210,30.0,210.0,t,0,0,0,117,41,5,f,f,f,moderate,f,2,1,1,0
2,none,f,1.0,1.0,Besiktas,41.07731,29.03891,t,6,1.0,2.0,5.0,$232.00,1,$0.00,21,1125,21,21,1125,1125,21.0,1125.0,t,0,0,0,2,0,0,f,t,f,strict_14_with_grace_period,f,1,1,0,0
3,none,f,1.0,1.0,Beyoglu,41.03220,28.98216,f,2,1.0,1.0,1.0,$267.00,2,$99.00,5,90,5,5,90,90,5.0,90.0,t,0,8,8,277,2,1,f,f,f,moderate,f,1,1,0,0
4,none,f,19.0,19.0,Sisli,41.04471,28.98567,t,5,1.0,1.0,3.0,$654.00,2,$66.00,3,360,3,3,360,360,3.0,360.0,t,12,34,45,302,0,0,f,t,f,moderate,f,19,19,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,none,f,1.0,1.0,Bahcelievler,40.99535,28.84067,t,4,NaN,1.0,2.0,$107.00,1,$0.00,3,1125,3,3,1125,1125,3.0,1125.0,t,29,59,89,269,0,0,f,t,f,flexible,f,1,0,1,0
17324,none,f,1.0,1.0,Sisli,41.05814,28.99161,f,3,1.0,2.0,2.0,$303.00,1,$0.00,3,15,3,3,15,15,3.0,15.0,t,23,29,41,131,0,0,f,f,f,flexible,f,1,1,0,0
17325,none,f,1.0,1.0,Maltepe,40.95070,29.12381,f,1,1.0,1.0,1.0,$148.00,1,$0.00,1,5,1,1,5,5,1.0,5.0,t,0,7,33,33,0,0,f,f,f,flexible,f,1,0,1,0
17326,none,f,1.0,1.0,Besiktas,41.07760,29.02340,t,2,1.0,1.0,1.0,$148.00,1,$0.00,1,7,1,1,7,7,1.0,7.0,t,29,49,79,115,0,0,f,f,f,moderate,f,1,0,1,0
